# Install 💻💻

In [1]:
!pip install azure-storage-blob

     |████████████████████████████████| 366 kB 2.0 MB/s            
You should consider upgrading via the '/Users/fabienbarrios/Desktop/Azure/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install azure-cosmos

     |████████████████████████████████| 215 kB 1.8 MB/s            
You should consider upgrading via the '/Users/fabienbarrios/Desktop/Azure/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install azure-datalake-store

     |████████████████████████████████| 61 kB 960 kB/s             
You should consider upgrading via the '/Users/fabienbarrios/Desktop/Azure/venv/bin/python -m pip install --upgrade pip' command.


# Imports 🔧🔧

In [5]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
from azure.cosmos import cosmos_client, exceptions
from azure.cosmos.partition_key import PartitionKey #  script Python perso où j'indique mes credentials Azure (à vs de créer le votre)

In [6]:
cosmosdb_uri = config.settings['cosmosdb_uri']
cosmosdb_primary_key = config.settings['cosmosdb_primary_key']
cosmosdb_database_id = config.settings['cosmosdb_database_id']
cosmosdb_container_id = config.settings['cosmosdb_container_id']

NameError: name 'config' is not defined

# Se connecter à Azure 🔑 ? 

* Après avoir installé le [client az](https://docs.microsoft.com/fr-fr/cli/azure/), vérifier sa version et se connecter ? 

In [9]:
!az version
!az version --output jsonc
!az version --output table
# S : 2.xx

{
  "azure-cli": "2.38.0",
  "azure-cli-core": "2.38.0",
  "azure-cli-telemetry": "1.0.6",
  "extensions": {}
}
{
  "azure-cli": "2.38.0",
  "azure-cli-core": "2.38.0",
  "azure-cli-telemetry": "1.0.6",
  "extensions": {}
}
Azure-cli    Azure-cli-core    Azure-cli-telemetry
-----------  ----------------  ---------------------
2.38.0       2.38.0            1.0.6


In [10]:
!az login

A web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "c371d4f5-b34f-4b06-9e66-517fed904220",
    "id": "fdc2db14-4f80-4cb7-ba6c-c410ef691d4f",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure for Students",
    "state": "Enabled",
    "tenantId": "c371d4f5-b34f-4b06-9e66-517fed904220",
    "user": {
      "name": "fbarrios@myges.fr",
      "type": "user"
    }
  }
]


* Q : Vérifier son compte et les locations prises en charge par mon abonnement ? 

In [11]:
# !az account show
# !az account show -o jsonc
# !az account list-locations
# !az account list-locations -o jsonc
# az account list-locations -o yaml
# az account list-locations -o tsv
#
# !az account list-locations -o table | grep France
# !az account list-locations --query "[].{Region:name}" --out table
# S : l'option --query permet de spécifier une Qry sur le JSON ; le langage est jmespath
# https://jmespath.org/

* Utiliser le [Cloud Shell](https://docs.microsoft.com/fr-fr/azure/cloud-shell/using-cloud-shell-editor)

In [ ]:
# Q : Ouvrir l'éditeur intégré (pr créer et modifier facilement des fichiers de config)   
code .
ls
python
pip
git

# Ressource Group ? 

<center><img src="./azure-rg.png"></center>
<center><img src="./azure-rg-lock.png" style="width:800px;height:300px;"></center>

* Q : Cmnt créer un groupe de ressources avec l'[interface](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/manage-resource-groups-portal#what-is-a-resource-group) et la [az-cli](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/manage-resource-groups-cli#create-resource-groups) ? 

In [12]:
!az group create --name gr-forma-fondamnt --location francecentral
# S : Vérifier que vs avez "Succeeded"  =>  "properties": { "provisioningState": "Succeeded" }

{
  "id": "/subscriptions/fdc2db14-4f80-4cb7-ba6c-c410ef691d4f/resourceGroups/gr-forma-fondamnt",
  "location": "francecentral",
  "managedBy": null,
  "name": "gr-forma-fondamnt",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [13]:
!az group show --name gr-forma-fondamnt

{
  "id": "/subscriptions/fdc2db14-4f80-4cb7-ba6c-c410ef691d4f/resourceGroups/gr-forma-fondamnt",
  "location": "francecentral",
  "managedBy": null,
  "name": "gr-forma-fondamnt",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [14]:
# Q : Cmnt afficher les groupes de ressources crées ? 
!az group list -o table

Name               Location       Status
-----------------  -------------  ---------
gr-forma-fondamnt  francecentral  Succeeded
gr-ia-poc          eastus         Succeeded


* Q : Cmnt mettre en place un verrou avec l'[interface](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/lock-resources?tabs=json#configure-locks) et la [az-cli](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/lock-resources?tabs=json#azure-cli) (pr empêcher la supp/modif inattentionnel du groupe de ressources) ? 

In [15]:
!az lock create -n lockGroup --lock-type CanNotDelete -g gr-forma-fondamnt

{
  "id": "/subscriptions/fdc2db14-4f80-4cb7-ba6c-c410ef691d4f/resourceGroups/gr-forma-fondamnt/providers/Microsoft.Authorization/locks/lockGroup",
  "level": "CanNotDelete",
  "name": "lockGroup",
  "notes": null,
  "owners": null,
  "resourceGroup": "gr-forma-fondamnt",
  "type": "Microsoft.Authorization/locks"
}


* Q : Obtenir tous les verrous d’un groupe de ressources ? 

In [16]:
!az lock list --resource-group gr-forma-fondamnt -o table 

Level         Name       ResourceGroup
------------  ---------  -----------------
CanNotDelete  lockGroup  gr-forma-fondamnt


* Q : Cmnt supprimer un groupe de ressources avec l'[interface]((https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/delete-resource-group?tabs=azure-portal#delete-resource-group)) et [az-cli](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/delete-resource-group?tabs=azure-cli#delete-resource-group)  ? => Impossible parce qu'il faut faire sauter le verrou avant ! 

In [13]:
!az group delete -n gr-forma-fondamnt --no-wait --yes 

ERROR: (ScopeLocked) The scope '/subscriptions/14c96ffb-9d7c-4aca-b872-adf1f19b250f/resourcegroups/gr-forma-fondamnt' cannot perform delete operation because following scope(s) are locked: '/subscriptions/14c96ffb-9d7c-4aca-b872-adf1f19b250f/resourceGroups/gr-forma-fondamnt'. Please remove the lock and try again.


* Q : Cmnt supp un verrou ? => 2 étapes sont nécessaires : 1) On réccupère l'`id` du verrou et 2) on supp le verrou

In [17]:
# 1) On réccupère d'id du verrou
!az lock show --name lockGroup --resource-group gr-forma-fondamnt  --output tsv --query id
# S : /subscriptions/14c96ffb-9d7c-4aca-b872-adf1f19b250f/resourceGroups/gr-forma-fondamnt/providers/Microsoft.Authorization/locks/lockGroup

/subscriptions/fdc2db14-4f80-4cb7-ba6c-c410ef691d4f/resourceGroups/gr-forma-fondamnt/providers/Microsoft.Authorization/locks/lockGroup


In [18]:
# 2) On supp le verrou
!az lock delete  --ids /subscriptions/14c96ffb-9d7c-4aca-b872-adf1f19b250f/resourceGroups/gr-forma-fondamnt/providers/Microsoft.Authorization/locks/lockGroup
# On vérifiie qu'il n'y a plus de vrrou
!az lock list --resource-group gr-forma-fondamnt 


[]


# Storage Account ? 

<center><img src="azure-storage.png"style="width:800px;height:300px;"></center>

* **TP 1 : Concevoir une infra de stockage adéquate aux besoins de l’entreprise** avec l'[interface](https://docs.microsoft.com/fr-fr/azure/storage/common/storage-account-create?toc=%2Fazure%2Fstorage%2Fblobs%2Ftoc.json&tabs=azure-portal#create-a-storage-account-1) et [az-cli](https://docs.microsoft.com/fr-fr/azure/storage/common/storage-account-create?toc=%2Fazure%2Fstorage%2Fblobs%2Ftoc.json&tabs=azure-cli#create-a-storage-account-1)
    - Vous êtes Dév Azure au profit d’un Site Web de surf sur plages où les utilisateurs chargent et partagent des photos et des vidéos pour se décider de la meilleure plage (pour surfer) : météo, nombre de surfers, … 
    - Donc : 
        -	Le site doit gérer des pics inattendus
        -	Le chargement des vidéos doit être rapide ! 
        -	Le contenu obsolète doit être supprimé  
        -	data stockées dans USA Ouest, 
        -	géo-redondant pour effectuer la réplication sur différents data center à travers le monde, 
        -	accessible uniquement par le biais de https 
        -	facturés sur l’abonnement du service commercial ?


* **TP 2 : Connecter l’Explorateur Stockage Azure à un compte de stockage** ([sandbox](https://docs.microsoft.com/fr-fr/learn/modules/upload-download-and-manage-data-with-azure-storage-explorer/3-exercise-connect-storage-account))

* **TP 3 : "`CRUD`" sur un storage account** avec le [portail](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-portal), [az-cli](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-cli) et [Python](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?tabs=environment-variable-windows) :
    - Charger (un et plusieurs files) et Lister le contenu d'un contenair
    - Télécharger (un et plusieurs files d'une façon synchrone et asynchrone)
    - Renommer (un et plusieurs files)
    - Déplacer les files d'un conteneur vers un autre
    - Calculer la taille d'un contenair ? Pour vous aider [az-cli](https://github.com/Azure-Samples/azure-cli-samples/blob/master/storage/calculate-container-size/calculate-container-size.sh)
    - Supprimer (un et plusieurs files)
    - Après avoir créee plusieurs contenairs qui commencent par un même préfix, coder un script `shell` `az-cli` qui permet d'automatiser la suppression de ces contenairs ? Pour vous aider [az-cli](https://github.com/Azure-Samples/azure-cli-samples/blob/master/storage/delete-containers-by-prefix/delete-containers-by-prefix.sh)

<center><img src="azure-storage1.png"style="width:800px;height:300px;"></center>

* **Correc**
    * Pour rappel : sur Azure vous pouvez bénéficier d'un compte de stockage gratuit et valable 12 mois
        - 5 Go d'objet blob de blocs chaud, localement redondant (LRS) 
        - 5 Go de Stockage de fichiers, LRS
        - 10 Go de stockage et de récupération d'archive, LRS/GRS, plus de 100 lectures

* Q : Créer un compte de stockage avec le portail voir pour les paramètres le document [TP_Correc-Créer un compte de stockage en utilisant le portail-WebAppSurf.pdf](https://1drv.ms/b/s!AmJGbSlW18YGsstdAOOZLduTnSCfbA?e=7s9GYe) et [az-cli-ttes les options](https://docs.microsoft.com/fr-fr/cli/azure/storage/account?view=azure-cli-latest#az-storage-account-create)

In [19]:
!az storage account create \
  --name storage \
  --resource-group gr-forma-fondamnt \
  --location francecentral \
  --sku Standard_LRS \
  --kind StorageV2 \
  --min-tls-version TLS1_2 \
  --access-tier Hot       

Resource provider 'Microsoft.Storage' used by this operation is not registered. We are registering for you.
Registration succeeded.
(StorageAccountAlreadyTaken) The storage account named storage is already taken.
Code: StorageAccountAlreadyTaken
Message: The storage account named storage is already taken.


* Q : Exporter le compte de stockage à partir d'un template avec le [portail](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/templates/export-template-portal#export-template-from-a-resource-group) et [az-cli](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/templates/export-template-cli#export-template-from-a-resource-group)

In [20]:
!az group export --name gr-forma-fondamnt

{- Starting ..
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {},
  "resources": [],
  "variables": {}
}


* Q : Déployer le compte de stockage à partir d'un template avec [az-cli](https://docs.microsoft.com/en-us/azure/azure-resource-manager/management/manage-resources-cli#deploy-a-template)

In [21]:
# Exemple générique
!az deployment group create --resource-group gr-forma-fondamnt --template-uri "https://raw.githubusercontent.com/Azure/azure-quickstart-templates/master/quickstarts/microsoft.storage/storage-account-create/azuredeploy.json"

 - Starting .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Running .. | Running .. - Finished ..
  "id": "/subscriptions/fdc2db14-4f80-4cb7-ba6c-c410ef691d4f/resourceGroups/gr-forma-fondamnt/providers/Microsoft.Resources/deployments/azuredeploy",
  "location": null,
  "name": "azuredeploy",
  "proper

In [22]:
# Notre storage account
!az deployment group create --resource-group gr-forma-fondamnt --template-uri "C:/Users/bejao/OneDrive/Azure/1-storageAccount/template-storage-webappsurf/template-storage-account.json"

Unable to retrieve url C:/Users/bejao/OneDrive/Azure/1-storageAccount/template-storage-webappsurf/template-storage-account.json


* Q : Upload, download, and list blobs with the [portail](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-portal), [az-cli](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-cli) et [Python](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?tabs=environment-variable-windows)

In [23]:
# 1 : Réccupérer la clé d'accès au compte de stockage
!az storage account keys list --resource-group gr-forma-fondamnt --account-name storagewebappsurf --query "[0].value" --output tsv
# K2g+bCyk1DgZTU4j4vSQQaXtNvx1mlQllCBsPpcEbzJ8OU/chfl1YLq+MftytpoN/27F9osV68bR+ASt9/1EFQ==

(ResourceNotFound) The Resource 'Microsoft.Storage/storageAccounts/storagewebappsurf' under resource group 'gr-forma-fondamnt' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.Storage/storageAccounts/storagewebappsurf' under resource group 'gr-forma-fondamnt' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix


In [24]:
# 2 : Créer un contenair
!az storage container create --name contenair-blob --account-name storagewebappsurf --account-key K2g+bCyk1DgZTU4j4vSQQaXtNvx1mlQllCBsPpcEbzJ8OU/chfl1YLq+MftytpoN/27F9osV68bR+ASt9/1EFQ==

<urllib3.connection.HTTPSConnection object at 0x10dd43dc0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known


In [25]:
# Vérifier les contenairs crées
!az storage container list --account-name storagewebappsurf -o table


There are no credentials provided in your command and environment, we will query for account key for your storage account.
It is recommended to provide --connection-string, --account-key or --sas-token in your command as credentials.

You also can add `--auth-mode login` in your command to use Azure Active Directory (Azure AD) for authorization if your login account is assigned required RBAC roles.
For more information about RBAC roles in storage, visit https://docs.microsoft.com/azure/storage/common/storage-auth-aad-rbac-cli.

In addition, setting the corresponding environment variables can avoid inputting credentials in your command. Please use --help to get more information about environment variable usage.

Skip querying account key due to failure: Storage account 'storagewebappsurf' not found.
<urllib3.connection.HTTPSConnection object at 0x1085272e0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known


In [26]:
# 3 Upload image
!az storage blob upload \
    --account-name storagewebappsurf \
    --container-name contenair-blob \
    --file "C:/Users/bejao/OneDrive/data/CognitiveServices/ComputerVision/smart-cam-supermarket-surveillance/store_cam1.jpg" \
    --name store_cam1.jpg


There are no credentials provided in your command and environment, we will query for account key for your storage account.
It is recommended to provide --connection-string, --account-key or --sas-token in your command as credentials.

You also can add `--auth-mode login` in your command to use Azure Active Directory (Azure AD) for authorization if your login account is assigned required RBAC roles.
For more information about RBAC roles in storage, visit https://docs.microsoft.com/azure/storage/common/storage-auth-aad-rbac-cli.

In addition, setting the corresponding environment variables can avoid inputting credentials in your command. Please use --help to get more information about environment variable usage.

Skip querying account key due to failure: Storage account 'storagewebappsurf' not found.
[Errno 2] No such file or directory: 'C:/Users/bejao/OneDrive/data/CognitiveServices/ComputerVision/smart-cam-supermarket-surveillance/store_cam1.jpg'
Please check the file path.


In [27]:
# List the blobs in a container
!az storage blob list \
    --account-name storagewebappsurf \
    --container-name contenair-blob \
    --output table \


There are no credentials provided in your command and environment, we will query for account key for your storage account.
It is recommended to provide --connection-string, --account-key or --sas-token in your command as credentials.

You also can add `--auth-mode login` in your command to use Azure Active Directory (Azure AD) for authorization if your login account is assigned required RBAC roles.
For more information about RBAC roles in storage, visit https://docs.microsoft.com/azure/storage/common/storage-auth-aad-rbac-cli.

In addition, setting the corresponding environment variables can avoid inputting credentials in your command. Please use --help to get more information about environment variable usage.

Skip querying account key due to failure: Storage account 'storagewebappsurf' not found.
<urllib3.connection.HTTPSConnection object at 0x111b585e0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known


In [28]:
# Manip avec Python
try:
    print("Azure Blob Storage v" + __version__)

    # Quick start code goes here

except Exception as ex:
    print('Exception:')
    print(ex)

Azure Blob Storage v12.12.0


In [29]:
# StorageAccountName = storagewebappsurf
# Key = K2g+bCyk1DgZTU4j4vSQQaXtNvx1mlQllCBsPpcEbzJ8OU/chfl1YLq+MftytpoN/27F9osV68bR+ASt9/1EFQ==
# Connectionstring = DefaultEndpointsProtocol=https;AccountName=storagewebappsurf;AccountKey=K2g+bCyk1DgZTU4j4vSQQaXtNvx1mlQllCBsPpcEbzJ8OU/chfl1YLq+MftytpoN/27F9osV68bR+ASt9/1EFQ==;EndpointSuffix=core.windows.net
# Taper dans un cmd la commande suivate : 
setx AZURE_STORAGE_CONNECTION_STRING "DefaultEndpointsProtocol=https;AccountName=storagewebappsurf;AccountKey=K2g+bCyk1DgZTU4j4vSQQaXtNvx1mlQllCBsPpcEbzJ8OU/chfl1YLq+MftytpoN/27F9osV68bR+ASt9/1EFQ==;EndpointSuffix=core.windows.net"

SyntaxError: invalid syntax (3546445987.py, line 5)

In [ ]:
# Créer un contenair
# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string("DefaultEndpointsProtocol=https;AccountName=storagewebappsurf;AccountKey=K2g+bCyk1DgZTU4j4vSQQaXtNvx1mlQllCBsPpcEbzJ8OU/chfl1YLq+MftytpoN/27F9osV68bR+ASt9/1EFQ==;EndpointSuffix=core.windows.net")

# Create a unique name for the container
container_name = str(uuid.uuid4())

# Create the container
container_client = blob_service_client.create_container(container_name)

In [ ]:
# Create a local directory 
local_path = "C:/Users/bejao/OneDrive/data/CognitiveServices/ComputerVision/smart-cam-supermarket-surveillance/store_cam1.jpg"

# Create a file in the local data directory to upload and download
# local_file_name = str(uuid.uuid4()) + ".txt"
# upload_file_path = os.path.join(local_path, local_file_name)

# Write text to the file
# file = open(local_path, 'w')
# file.write("store_cam1.jpg")
# file.close()

# Create a blob client using the local file name as the name for the blob
blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_path)

print("\nUploading to Azure Storage as blob:\n\t" + local_path)

# Upload the created file
with open(local_path, "rb") as data:
    blob_client.upload_blob(data)

In [ ]:
# liste des objets blob d’un conteneur
print("\nListing blobs...")

# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print("\t" + blob.name)

In [ ]:
# Download the blob to a local file
# Add 'DOWNLOAD' before the .txt extension so you can see both files in the data directory
download_file_path = os.path.join(local_path, str.replace(local_path ,'.txt', 'DOWNLOAD.txt'))
print("\nDownloading blob to \n\t" + download_file_path)

with open(download_file_path, "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())

* Q : Déplacer le compte de stockage vers une autre région avec le [portail](https://docs.microsoft.com/fr-fr/azure/storage/common/storage-account-move?tabs=azure-portal)

* Q : Supp le compte de stockage

In [ ]:
!az storage account delete --name <storage-account> --resource-group <resource-group>

# WebApp ? 

<center><img src="azure-webapp.png"style="width:800px;height:300px;"></center>

* **TP : Déployer une [application web](https://challenge-48h.web.app/form.html) qui stocke et affiche des images à partir du Stockage Azure le tout en mettant en place une IA** : suivez ce [guide](https://docs.microsoft.com/fr-fr/azure/storage/blobs/storage-upload-process-images?tabs=javascript%2Cazure-cli)  
    <br>
* Pour vous aider, répondre progressivement aux questions suivantes :  
    <br>
    - **Déployer la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante sur Azure à partir d'un git local, de github, à partir de Docker, ... :**  
    
        - Q : Déployer la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante sur Azure à partir d'un **git local** avec le [portail](https://docs.microsoft.com/fr-fr/azure/app-service/quickstart-python?) et l'automatiser avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/quickstart-python?tabs=flask%2Cwindows%2Cazure-cli%2Cterminal-bash%2Cvscode-deploy%2Cdeploy-instructions-azportal%2Cdeploy-instructions-zip-azcli#2---create-a-web-app-in-azure)
        - Q : Déployer la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante sur Azure à partir d'un github (sans déploiment continu) avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/scripts/cli-deploy-github?toc=/cli/azure/toc.json)
        - Q : Déployer la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante sur Azure à partir d'un github **avec déploiment continu** avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/scripts/cli-continuous-deployment-github?toc=/cli/azure/toc.json)
        - Q : Déployer la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante sur Azure à l'aide d'**une image Docker** avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/scripts/cli-linux-docker-aspnetcore?toc=/cli/azure/toc.json)
        - Déployer la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante sur Azure dans un environnement de **préprod** avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/scripts/cli-deploy-staging-environment?toc=/cli/azure/toc.json)  
    <br>
    - **Connecter la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) à une DB, à un compte de stockage, à CosmosDB :**  
        - Connecter la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante à un compte de stockage avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/scripts/cli-connect-to-storage?toc=/cli/azure/toc.json) et Augmenter le code de la webapp en y incluant la connexion et le CRUD ?  
        - Connecter la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante à une DB SQL avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/scripts/cli-connect-to-sql?toc=/cli/azure/toc.json) ? 
        - Connecter la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante à CosmosDB avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/scripts/cli-connect-to-documentdb?toc=/cli/azure/toc.json) ? 
        - Connecter la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante à un cache redis avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/scripts/cli-connect-to-redis?toc=/cli/azure/toc.json) ?  
    <br>
    - **Configurer la WebApp :**  
        - Déployer la [WebApp](https://github.com/Azure-Samples/msdocs-python-flask-webapp-quickstart) suivante sur Azure avec un point de terminaison privé avec [az-cli](https://docs.microsoft.com/fr-fr/azure/app-service/scripts/cli-deploy-privateendpoint?toc=/cli/azure/toc.json)
        - Personaliser le nom de domaine de la webapp avec le [portail](https://docs.microsoft.com/fr-fr/azure/app-service/app-service-web-tutorial-custom-domain?tabs=a%2Cazurecli#4-enable-the-mapping-in-your-app) et le [sécuriser](https://docs.microsoft.com/fr-fr/azure/app-service/configure-ssl-bindings?source=docs#secure-a-custom-domain) (Att, c'est payant) 

<center><img src="azure-passionfroid.png"style="width:900px;height:400px;"></center>

* **Correc**

* Create an `appservice` plan (a machine) where your site will run

In [ ]:
!az appservice plan create --name ASP-grformafondamnt-88fc --location francecentral --is-linux --sku F1 --resource-group gr-forma-fondamnt


* Create the web application

In [ ]:
az webapp create --name webappsurf --plan ASP-grformafondamnt-88fc --runtime PYTHON|3.7 --resource-group gr-forma-fondamnt

Q : Automatiser 

In [ ]:
C:/Users/bejao/OneDrive/Azure/2-webapp/deployment-webapp.sh

# VM Linux ? 

* **TP individuel :** 

    - Votre entreprise a réservé un stand dans un salon important consacré à Linux. Vous prévoyez une zone de démonstration contenant trois terminaux, chacun connecté à une VM Linux distincte. À la fin de chaque journée, vous souhaitez supprimer les VM et les recréer, afin qu’elles soient réinitialisées pour le lendemain matin. Créer les VM manuellement alors que vous êtes fatigué, constituerait une source d’erreurs potentielle. Vous voulez donc écrire un script (PowerShell, az-cli, bicep) pour automatiser le processus de création et de suppression des VM : déclaration de var, boucle for, …

* **TP Collectif :**  
    <br>
    - Créer un groupe de ressources
    - Autoriser deux users à créer/gérer des VM (dans le groupe de ressources déjà crée)  
    - **Le 1è user installera :**   
    
        -	Une VM Linux Ubuntu 18.04 LTS (Standard D2s v3), sans redondance   
        -	S’y connectera avec `SSH`, `HTTP` et `HTTPS`   
        -	Installera dessus `NGINX` with [Portal](https://docs.microsoft.com/fr-fr/azure/virtual-machines/linux/quick-create-portal), [CLI](https://docs.microsoft.com/fr-fr/azure/virtual-machines/linux/quick-create-cli)  
    <br>    
    - **Le 2è user installera :**   
    
        - Une VM Windows Server avec `RDP`, `SSH`, `HTTP` et `HTTPS`  
        - S’y connectera avec `RDP`, `SSH`, `HTTP` et `HTTPS`  
        - NB : pour que la VM soit accessible à distance, elle doit être crée avec une adresse ip publique, with [Portal](https://docs.microsoft.com/fr-fr/azure/virtual-machines/windows/quick-create-portal), [CLI](https://docs.microsoft.com/fr-fr/azure/virtual-machines/windows/quick-create-cli)  
    <br>    
    - Vous vous connectez/prendrez le contrôle de la VM à partir de votre machine  
    - Vous vous accordez l’accès à la VM aux membres de votre groupe avec des droits différents  
    - Mettre un verrou sur le groupe de ressources et cherchez à supp le groupe de ress, with [Portal](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/lock-resources?tabs=json#configure-locks), [CLI](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/manage-resource-groups-cli#lock-resource-groups)  
    -  
    - Faire sauter le verou,   
    - Redimensionner la VM  
    - Exporter le modèle de déploiement ARM : with [Portal](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/templates/export-template-portal#export-template-from-a-resource-group), [CLI](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/templates/export-template-cli#export-template-from-a-resource-group),   
    - Supprimer le groupe de ress (pour arrêter la facturation) : with [Portal](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/delete-resource-group?tabs=azure-portal#delete-resource-group), [CLI](https://docs.microsoft.com/fr-fr/azure/azure-resource-manager/management/delete-resource-group?tabs=azure-cli#delete-resource-group)  
    - Redéployer le modèle exporté  


# [CosmosDB](https://azure.microsoft.com/en-us/try/cosmosdb/) ? 

<center><img src="./azure-cosmosdb.png" "></center>

* We'll create a `database` and `containers` with the Portal ([help1](https://docs.microsoft.com/fr-fr/azure/cosmos-db/sql/create-cosmosdb-resources-portal), [help2](https://www.sqlshack.com/querying-json-documents-in-azure-cosmos-db-using-the-sql-api/)), [az-cli](https://docs.microsoft.com/fr-fr/azure/cosmos-db/scripts/cli/sql/create), Python ([help1](https://docs.microsoft.com/fr-fr/azure/cosmos-db/sql/create-sql-api-python), [help2](https://docs.microsoft.com/en-us/python/api/overview/azure/cosmos-readme?view=azure-python#authenticate-the-client)), [Spark](https://docs.microsoft.com/fr-fr/azure/cosmos-db/sql/create-sql-api-spark), import data (_events that occurred on e-commerce website - e.g. a user viewing an item, adding it to their cart, or purchasing it_) and run some queries over it ? 
* We will [partition](https://docs.microsoft.com/azure/cosmos-db/partition-data) by `CartId`, which represents the individual cart of each user. This will give us an even distribution of throughput and storage in our container. 
* [guide de survie](https://1drv.ms/b/s!AmJGbSlW18YGs8spKHX4fbgd5gy8Ug?e=Yw9fOs)

<center><img src="./azure-cosmos-db.png" style="width:800px;height:300px;"></center>

In [15]:
# from azure.cosmos import CosmosClient
# from azure.cosmos.partition_key import PartitionKey
# import config
import json

In [27]:
# Initialize the Cosmos client
# client = CosmosClient(cosmosdb_uri, cosmosdb_primary_key)
# print("Connextion success to", type(client))

In [7]:
db = client.create_database_if_not_exists(cosmosdb_database_id)
print('Database created or used', type(db))

Database created or used <class 'azure.cosmos.database.DatabaseProxy'>


In [30]:
# Get database propreties
print(client.get_database_client(cosmosdb_database_id))
properties = db.read()
print(json.dumps(properties))

<DatabaseProxy [dbs/SampleDB]>
{"id": "SampleDB", "_rid": "FP1RAA==", "_self": "dbs/FP1RAA==/", "_etag": "\"00004604-0000-0700-0000-624566170000\"", "_colls": "colls/", "_users": "users/", "_ts": 1648715287}


In [8]:
# Les conteneurs sont une collection d’éléments (des documents JSON) qui peuvent avoir différents schémas
# Using a good partition key improves the performance of database operations.
container = db.create_container_if_not_exists(id='WebsiteData', partition_key=PartitionKey(path='/CartID'))
print('Container WebsiteData created', type(container))

Container WebsiteData created <class 'azure.cosmos.container.ContainerProxy'>


* Load [website_transactions.json](https://1drv.ms/u/s!AmJGbSlW18YGs8sqOV7HS6cPWsaVBQ?e=tZmnKQ) and insert into the container
* Remarquez que chaque document de `website_transactions.json` comprend un champs `id` (obligatoire) 

In [20]:
with open('C:/Users/bejao/OneDrive/data/website_transactions.json') as f:
    data = f.read()
    data = json.loads(data)
    
for i in range(len(data)) : 
    container.create_item(data[i])

* Load [website_transactions.json](https://1drv.ms/u/s!AmJGbSlW18YGs8sqOV7HS6cPWsaVBQ?e=tZmnKQ) and insert into a new container `family`

In [ ]:
# <create_container_if_not_exists>
container_name = 'FamilyContainer'
container = database.create_container_if_not_exists(
    id=container_name, 
    partition_key=PartitionKey(path="/lastName"),
    offer_throughput=400

for i in range(1, 10):
    container.upsert_item({
            'id': 'item{0}'.format(i),
            'productName': 'Widget',
            'productModel': 'Model {0}'.format(i)
        }
    )

The new database and container should show up under the **Data** section. Use the refresh icon after completing the previous cell. 

![Refresh Data resource tree to see newly created resources](https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/refreshData.png)

In [ ]:
# Query these items using the SQL query syntax

query = "SELECT c.Action, c.Price as ItemRevenue, c.Country, c.Item FROM WebsiteData AS c"

items = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

request_charge = container.client_connection.last_response_headers['x-ms-request-charge']

print('Query returned {0} items. Operation consumed {1} request units'.format(len(items), request_charge))

In [24]:
items

[{'Action': 'Viewed',
  'ItemRevenue': 9,
  'Country': 'Tunisia',
  'Item': 'Black Tee'},
 {'Action': 'Viewed',
  'ItemRevenue': 19.99,
  'Country': 'Antigua and Barbuda',
  'Item': 'Flannel Shirt'},
 {'Action': 'Added',
  'ItemRevenue': 3.75,
  'Country': 'Guinea-Bissau',
  'Item': 'Socks'},
 {'Action': 'Viewed',
  'ItemRevenue': 3.75,
  'Country': 'Guinea-Bissau',
  'Item': 'Socks'},
 {'Action': 'Viewed',
  'ItemRevenue': 55,
  'Country': 'Czech Republic',
  'Item': 'Rainjacket'},
 {'Action': 'Viewed',
  'ItemRevenue': 350,
  'Country': 'Iceland',
  'Item': 'Cosmos T-shirt'},
 {'Action': 'Added',
  'ItemRevenue': 19.99,
  'Country': 'Syrian Arab Republic',
  'Item': 'Button-Up Shirt'}]

In [22]:
# Run a query using the built-in Azure Cosmos notebook magic
%%sql
SELECT c.Action, c.Price as ItemRevenue, c.Country, c.Item FROM Website AS c

SyntaxError: invalid syntax (Temp/ipykernel_6044/3214461879.py, line 3)

In [ ]:
# Read items with full Python (key value lookups by partition key and id, aka point reads)
for family in family_items_to_create:
    item_response = container.read_item(item=family['id'], partition_key=family['lastName'])
    request_charge = container.client_connection.last_response_headers['x-ms-request-charge']
    print('Read item with id {0}. Operation consumed {1} request units'.format(item_response['id'], (request_charge)))

In [ ]:
person = {
    "id": "1",
    "category": "personal",
    "name": "groceries",
    "description": "Pick up apples and strawberries.",
    "isComplete": false
}

In [ ]:
# Delete 
# The SQL API in Cosmos DB does not support the SQL DELETE statement.
# To delete items from a container, use ContainerProxy.delete_item. 

for item in container.query_items(
        query='SELECT * FROM products p WHERE p.productModel = "Model 2"',
        enable_cross_partition_query=True):
    container.delete_item(item, partition_key='Widget')

# [Data Lake](https://azure.microsoft.com/en-us/solutions/data-lake/) ? 

* Créer un datalake avec le [portal](https://docs.microsoft.com/fr-fr/azure/storage/blobs/create-data-lake-storage-account)
* [TP](https://docs.microsoft.com/fr-fr/azure/storage/blobs/data-lake-storage-use-databricks-spark) : Data Lake, Spark, Databricks
* [TP](https://docs.microsoft.com/fr-fr/azure/storage/blobs/data-lake-storage-tutorial-extract-transform-load-hive) : ETL Hive

In [ ]:
import azure.datalake.store